# Introduction
**Inverse Kinematics** is the process of knowing the end-point of the desired articulation and solving for the joint parameters. This is a fundamental problem in robotic control. This involves finding the solution to a set of nonlinear equations, and thus suffers from its perils: solutions may not exist and may not be unique. Think of it this way: an arm can only reach points within its **workspace** (so any point outside has no solution); and within the workspace, the elbow, wrist, and sholder can flex while your fingertips remain touching the same surface.

Because of this, there are two methods to solve for $\rho$: an **analytic technique** or an iterative **gradient-descent search**.

# Analytic Technique
Solving through analysis is more mathematics intensive, involving a comprehensive understanding of both linear algebra and trigonometry. This is usually helpful in low DOF mechanisms. As the DOF rises, so too does the computational complexity of the solutions, and thus becomes more succeptible to computational error.

In a sense, the final transformation is given (that is, solved first), so we need to solve for each intermediate articulation. This can be done (using DH convention) by pre- and post-multiplying the inverse transformations of the articulations to isolate specific unknowns. Consider the following relationships:

$$
T_4^0 = A_1 A_2 A_3 A_4 \\
A_3^{-1} A_2^{-1} A_1^{-1} T_4^0 = A_4 \\
T_4^0 A_4^{-1}A_3^{-1}A_2^{-1} = A_1
$$

Notice the pattern there? Once as we isolate those end-transformations, The others become just as simple:

$$
A_2^{-1}A_1^{-1}T_4^0 = A_3 A_4 \\
A_2^{-1}A_1^{-1}T_4^0 A_4^{-1} = A_3
$$

Because of this, we need each articulation's *inverse transformation*.

## Example
![Three Link Example](images/ThreeLink.png)

Let's consider the three-link planar manipulator again. We'll begin by assuming the forward kinematic solution is known, but only the values of the matrix. To simplify things, we'll consider this matrix:

$$
T_4^0 = A_1 A_2 A_3 A_4 = 
\begin{bmatrix}
r_{11} & r_{12} & 0 & p_x \\
r_{21} & r_{22} & 0 & p_y \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

Recall when we solved the forward solution, we arrived at the form:

$$
\begin{bmatrix}
c_{123} & -s_{123} & 0 & (c_{123}L_3 + c_{12}L_2 + c_1 L_1) \\
s_{123} & c_{123} & 0 & (s_{123}L_3 + s_{12}L_2 + s_1 L_1) \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

We'll also need each articulation's inverse transformation (using the same $s_k$ and $c_k$ as earlier):

$$
A_1^{-1} = \begin{bmatrix}c_1 & s_1 & 0 & 0 \\ -s_1 & c_1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}
A_2^{-1} = \begin{bmatrix}c_2 & s_2 & 0 & -c_2 L_1 \\ -s_2 & c_2 & 0 & s_2 L_1\\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix} \\
A_3^{-1} = \begin{bmatrix}c_3 & s_3 & 0 & -c_3 L_2 \\ -s_3 & c_3 & 0 & s_3 L_2 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}
A_4^{-1} = \begin{bmatrix}1 & 0 & 0 & -L_3 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}
$$

One *intermediate* variable we can solve for is $\Psi_{123}$; this represents total angle from the center point of the first articulation to the end manipulator point. This is simply $\Psi_{123} = \arctan2(r_{21}, r_{11})$. Now, as we solve for other rotary angles, we can use this value to find the remaining.

Let's try to isolate some more variables. In this case, let's look at the resulting equation from $T_4^0 A_4^{-1} = A_1 A_2 A_3$:

$$
\begin{bmatrix}
r_{11} & r_{12} & 0 & p_x - r_{11}L_3\\
r_{21} & r_{22} & 0 & p_y - r_{21}L_3\\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix} =
\begin{bmatrix}
c_{123} & -s_{123} & 0 & c_1(c_2 L_2 + L_1) - s_1(s_2 L_2) \\
s_{123} & c_{123} & 0 & s_1(c_2 L_2 + L_1) + c_1(s_2 L_2) \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
=
\begin{bmatrix}
c_{123} & -s_{123} & 0 & c_{12} L_2 + c_1 L_1 \\
s_{123} & c_{123} & 0 & s_{12} L_2 + s_1 L_1 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

Here, we have two pairs of elements with interesting equality:

$$
k_1 = p_x - r_{11}L_3 = c_{12} L_2 + c_1 L_1 \\
k_2 = p_y - r_{21}L_3 = s_{12} L_2 + s_1 L_1
$$

When we square and sum these, we can begin reducing by trigonometric identities:
$$
k_1^2 + k_2^2 = L_2^2 + L_1^2 + 2 L_2 L_1 (c_1 c_{12} + s_1 s_{12}) = L_2^2 + L_1^2 + 2 L_2 L_1 c_2
$$

With $c_2$ isolated, we can now solve for $Psi_2$:

$$
\Psi_2 = \arccos \left[{\frac{(k_1^2 + k_2^2) - (L_1 + L_2)}{2 L_1 L_2}}\right]
$$

This solution, however, implies that there are two solutions for this angle, symetric about zero. This corresponds to elbow-up and elbow-down configurations.

We'll skip solving for $\Psi_1$ for now... more trig identities can pull this from $k_1$ and $k_2$.

# Gradient-Descent Technique
To make this possible, we need some function which we are attempting to minimize. Let's use *distance from target position*, which corresponds to the error of $\rho_i$. In this way, we'll solve for the gradient of each articulation angle $\alpha_k \in \rho$, and when this gradient is very small, we'll say that the search is done.

$$
T^* = \begin{bmatrix}x^* & y^* & z^* & 1\end{bmatrix}^{\top} \\
T(\rho_i) = \begin{bmatrix}
c_{123} & -s_{123} & 0 & (c_{123}L_3 + c_{12}L_2 + c_1 L_1) \\
s_{123} & c_{123} & 0 & (s_{123}L_3 + s_{12}L_2 + s_1 L_1) \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix} \cdot
\begin{bmatrix} x_0 \\ y_0 \\ z_0 \\ 1 \end{bmatrix}\\
f(\rho_i) = (T^* - T(\rho_i))^2
$$

At this stage, we calculate $\nabla{\rho} f(\rho_i)$.